### **AQI-SupportVector Regressor Method**

In [5]:
# from google.colab import drive
# drive.mount('/content/drive/')

### **Import necessary libraries**

In [6]:
import pickle
import numpy as np
import pandas as pd


### **Loading cleaned data**

Before applying linear regression, I did feature engineerings such as dealing with outliers, null values and correlation analysis. The Cleland data has been saved.

In [15]:
from pathlib import Path
file_path = Path('D:\Internship\IBM Internship\Project1\Air-Quality-Index-prediction_with_deployment\ML_Applied_Word\PreProcess\df_cleaned_data.pkl')
with open(file_path,'rb') as file:
    df= pickle.load(file)

<>:2: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\I'
C:\Users\ruchi\AppData\Local\Temp\ipykernel_62332\2755762934.py:2: SyntaxWarning: invalid escape sequence '\I'
  file_path = Path('D:\Internship\IBM Internship\Project1\Air-Quality-Index-prediction_with_deployment\ML_Applied_Word\PreProcess\df_cleaned_data.pkl')


In [16]:
#Read value
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,10.8,17.0,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,10.8,17.0,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,10.8,17.0,4.3,1019.4,82.0,0.6,4.8,11.1,154.037500
3,10.8,17.0,4.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [17]:
#Seperating dependent and independent variables 
x=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [19]:
#Train-Test Split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=100)

### **SVR  Implementation without Hyperparameter Tuning**

In [20]:
from sklearn import svm
from sklearn import metrics
svr= svm.SVR(kernel='rbf')
svr.fit(x_train, y_train)

SVR()

In [21]:
pred_svr=svr.predict(x_test)

In [22]:
print('MAE:', metrics.mean_absolute_error(y_test,pred_svr))
print('MSE:', metrics.mean_squared_error(y_test, pred_svr))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test,pred_svr)))

MAE: 65.10972021385003
MSE: 7800.071167034752
RMSE: 88.31801156635464


Without any hyperparameter tuning support vector regressor RMSE value around 94 but in the linear regressor, we got around 57 So, next step we are going to do some hyperparameter tuning to find optimal parameters.

**SVR-Linear Kernel Implementation with hyperparameter tuning**

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
svr_tune = svm.SVR(kernel='linear')
param_grid_linear= {'C': [4,5,6,7,8,9,10,11,12,13,14,15]} 
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=100)
svr_linear= GridSearchCV(svr_tune, param_grid_linear,cv=cv, refit = True, verbose = 3)
svr_linear.fit(x_train, y_train)

Fitting 30 folds for each of 12 candidates, totalling 360 fits
[CV 1/30] END ..............................C=4;, score=0.607 total time=   0.0s
[CV 2/30] END ..............................C=4;, score=0.540 total time=   0.0s
[CV 3/30] END ..............................C=4;, score=0.413 total time=   0.0s
[CV 4/30] END ..............................C=4;, score=0.485 total time=   0.0s
[CV 5/30] END ..............................C=4;, score=0.461 total time=   0.0s
[CV 6/30] END ..............................C=4;, score=0.531 total time=   0.0s
[CV 7/30] END ..............................C=4;, score=0.561 total time=   0.0s
[CV 8/30] END ..............................C=4;, score=0.574 total time=   0.0s
[CV 9/30] END ..............................C=4;, score=0.575 total time=   0.0s
[CV 10/30] END .............................C=4;, score=0.506 total time=   0.0s
[CV 11/30] END .............................C=4;, score=0.432 total time=   0.1s
[CV 12/30] END .............................C=

GridSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=100),
             estimator=SVR(kernel='linear'),
             param_grid={'C': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]},
             verbose=3)

### **Model Evaluation**

In [26]:
from sklearn import metrics
rf_prediction=svr_linear.predict(x_test)
print(svr_linear.best_params_)
print(svr_linear.best_score_)
print('MAE:', metrics.mean_absolute_error(y_test, rf_prediction))
print('MSE:', metrics.mean_squared_error(y_test, rf_prediction))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, rf_prediction)))

{'C': 14}
0.5213607294593076
MAE: 40.3738719321604
MSE: 3145.0464829326875
RMSE: 56.08071400163061


In [27]:
#Save trained model
import pickle
# open a file, where you want to store the data
file = open('AQI_SVR_liner.pkl', 'wb')

# dump information to that file
pickle.dump(svr_linear
            , file)

So, Now we got RMSE values around 61 with hyperparameter tuning but without hyperparameter tuning, we got around 94. So right hyperparameter tuning will impact the RMSE. Further, Next step we are going to see,l How does  Support vector Polunomial kernel work with this data?

### **SVR-polynomial- Kernal Implementation**

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
svr_tune = svm.SVR(kernel='poly')
param_grid_poly=  {'degree': [2, 5,8,10,12], 'C': [10000,20000,30000,40000], 'coef0': [0,0.5,0.75,1]} 
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=100)
svr_poly= GridSearchCV(svr_tune, param_grid_poly,cv=cv, refit = True, verbose = 3)
svr_poly.fit(x_train, y_train)

Fitting 30 folds for each of 80 candidates, totalling 2400 fits
[CV 1/30] END .......C=10000, coef0=0, degree=2;, score=0.522 total time=   0.0s
[CV 2/30] END .......C=10000, coef0=0, degree=2;, score=0.467 total time=   0.0s
[CV 3/30] END .......C=10000, coef0=0, degree=2;, score=0.408 total time=   0.0s
[CV 4/30] END .......C=10000, coef0=0, degree=2;, score=0.368 total time=   0.0s
[CV 5/30] END .......C=10000, coef0=0, degree=2;, score=0.372 total time=   0.0s
[CV 6/30] END .......C=10000, coef0=0, degree=2;, score=0.443 total time=   0.0s
[CV 7/30] END .......C=10000, coef0=0, degree=2;, score=0.430 total time=   0.0s
[CV 8/30] END .......C=10000, coef0=0, degree=2;, score=0.460 total time=   0.0s
[CV 9/30] END .......C=10000, coef0=0, degree=2;, score=0.453 total time=   0.0s
[CV 10/30] END ......C=10000, coef0=0, degree=2;, score=0.466 total time=   0.0s
[CV 11/30] END ......C=10000, coef0=0, degree=2;, score=0.345 total time=   0.0s
[CV 12/30] END ......C=10000, coef0=0, degree

GridSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=100),
             estimator=SVR(kernel='poly'),
             param_grid={'C': [10000, 20000, 30000, 40000],
                         'coef0': [0, 0.5, 0.75, 1],
                         'degree': [2, 5, 8, 10, 12]},
             verbose=3)

In [32]:
from sklearn import metrics
poly_prediction=svr_poly.predict(x_test)
print(svr_poly.best_params_)
print(svr_poly.best_score_)
print('MAE:', metrics.mean_absolute_error(y_test, poly_prediction))
print('MSE:', metrics.mean_squared_error(y_test, poly_prediction))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, poly_prediction)))

{'C': 40000, 'coef0': 0.5, 'degree': 12}
0.5514632758849292
MAE: 37.69521678399911
MSE: 2911.321670132557
RMSE: 53.9566647424816


In [ ]:
#Save trained model
import pickle
# open a file, where you want to store the data
file = open('AQI_SVR_ploy.pkl', 'wb')

# dump information to that file
pickle.dump(svr_poly
            , file)

### **Conclusion**

###For SVR-Polynomial we got RMSE values around 57.2 but For SVR-Linear we got an RMSE value of 61 with hyperparameter tuning. So right hyperparameter tuning will impact the RMSE. Further, in the Next step we are going to see, How does Extra TreeRegressor work with this data?